# **SSCS - Test Playground - tf.data**

Developed by André Paiva

Based on SSCS Dataset created by Helena Cuesta and Emilia Gómez

In [ ]:
EXECUTE_ON_COLAB = False

In [ ]:
import os
import pandas as pd
import h5py
import json
import math
import zipfile
import tables
import requests
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy, Precision
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
num_threads = 6
os.environ["OMP_NUM_THREADS"] = str(num_threads)
os.environ["TF_NUM_INTRAOP_THREADS"] = str(num_threads)
os.environ["TF_NUM_INTEROP_THREADS"] = str(num_threads)

tf.config.threading.set_inter_op_parallelism_threads(num_threads)
tf.config.threading.set_intra_op_parallelism_threads(num_threads)
tf.config.set_soft_device_placement(True)

## 2 - Neural Network Model (VoasCNN)

In [ ]:
SAVE_MODEL = True
LOAD_MODEL = True
SPLIT_SIZE = 128
BATCH_SIZE = 32

In [ ]:
def voas_cnn_model():
    x_in = Input(shape=(360, SPLIT_SIZE, 1))
    
    x = BatchNormalization()(x_in)

    x = Conv2D(filters=32, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv1")(x)

    x = BatchNormalization()(x)

    x = Conv2D(filters=32, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv2")(x)

    x = BatchNormalization()(x)

    x = Conv2D(filters=16, kernel_size=(70, 3), padding="same",
        activation="relu", name="conv_harm_1")(x)

    x = BatchNormalization()(x)

    x = Conv2D(filters=16, kernel_size=(70, 3), padding="same",
        activation="relu", name="conv_harm_2")(x)

    ## start four branches now

    x = BatchNormalization()(x)

    ## branch 1
    x1a = Conv2D(filters=16, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv1a")(x)

    x1a = BatchNormalization()(x1a)

    x1b = Conv2D(filters=16, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv1b")(x1a)

    ## branch 2
    x2a = Conv2D(filters=16, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv2a")(x)

    x2a = BatchNormalization()(x2a)

    x2b = Conv2D(filters=16, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv2b")(x2a)

    ## branch 3

    x3a = Conv2D(filters=16, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv3a")(x)

    x3a = BatchNormalization()(x3a)

    x3b = Conv2D(filters=16, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv3b")(x3a)

    x4a = Conv2D(filters=16, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv4a")(x)

    x4a = BatchNormalization()(x4a)

    x4b = Conv2D(filters=16, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv4b"
    )(x4a)


    y1 = Conv2D(filters=1, kernel_size=1, name='conv_soprano',
                padding='same', activation='sigmoid')(x1b)
    y1 = tf.squeeze(y1, axis=-1)
    y2 = Conv2D(filters=1, kernel_size=1, name='conv_alto',
                padding='same', activation='sigmoid')(x2b)
    y2 = tf.squeeze(y2, axis=-1)
    y3 = Conv2D(filters=1, kernel_size=1, name='conv_tenor',
                padding='same', activation='sigmoid')(x3b)
    y3 = tf.squeeze(y3, axis=-1)
    y4 = Conv2D(filters=1, kernel_size=1, name='conv_bass',
                padding='same', activation='sigmoid')(x4b)
    y4 = tf.squeeze(y4, axis=-1)

    out = [y1, y2, y3, y4]

    model = Model(inputs=x_in, outputs=out, name='voasCNN')

    return model

## 3 - Auxiliar functions and Variables

Functions designed to manipulate the SSCS dataset.

### 3.1 - File path variables

In [ ]:
if(EXECUTE_ON_COLAB):
    dataset_dir = "/content/Datasets/"
else:
    dataset_dir = "Datasets/"
zipname = dataset_dir + "SSCS_HDF5.zip"
sscs_dir = dataset_dir + "SSCS_HDF5/"

songs_dir = sscs_dir + "sscs/"
splitname = sscs_dir + "sscs_splits.json"

### 3.2 - Download/Extract Scripts

In [ ]:
def download(url, fname):
    
    resp = requests.get(url, stream=True)
    total = int(resp.headers.get('content-length', 0))
    downloaded_size = 0
    with open(fname, 'wb') as file:
        for data in resp.iter_content(chunk_size=max(4096, int(total/10000))):
            size = file.write(data)
            downloaded_size += size
            percent = min(downloaded_size/total, 1.0)
            print(f"\r{percent:.2%} downloaded", end='')
            
    print()

In [ ]:
def sscs_download():
    
    if(not os.path.exists(dataset_dir)):
        os.mkdir(dataset_dir)
   
    if(not os.path.exists(zipname)):
        print("Downloading SSCS Dataset...")
        url = "https://github.com/Xornotor/SSCS_HDF5/releases/download/v1.0/SSCS_HDF5.zip"
        download(url, zipname)
    else:
        print("SSCS Dataset found.")

    if(not os.path.exists(sscs_dir)):
        print("Extracting SSCS Dataset...")
        with zipfile.ZipFile(zipname) as zf:
            os.mkdir(sscs_dir)
            zf.extractall(path=sscs_dir)
    else:
        print("SSCS Dataset already extracted.")
    

    print("Done.")

### 3.3 - Splits, songnames and songlists

In [ ]:
def sscs_get_split(split='train'):
    
    if(split.lower() == 'train' or split.lower() == 'validate' or
       split.lower() == 'test'):
        return json.load(open(splitname, 'r'))[split.lower()]
    else:
        raise NameError("Split should be 'train', 'validate' or 'test'.")

In [ ]:
def sscs_pick_songlist(first=0, amount=5, split='train'):
    
    songnames = sscs_get_split(split)
    return songnames[first:first+amount]

In [ ]:
def sscs_pick_random_song(split='train'):
    
    songnames = sscs_get_split(split)
    rng = np.random.randint(0, len(songnames))
    return songnames[rng]

In [ ]:
def sscs_pick_multiple_random_songs(amount, split='train'):
    
    return [sscs_pick_random_song() for i in range(amount)]

### 3.4 - Plots

In [ ]:
def sscs_plot(dataframe):

    aspect_ratio = (3/8)*dataframe.shape[1]/dataframe.shape[0]
    fig, ax = plt.subplots(figsize=(13, 7))
    im = ax.imshow(dataframe, interpolation='nearest', aspect=aspect_ratio,
        cmap = mpl.colormaps['BuPu'])
    ax.invert_yaxis()
    plt.show()

## 4 - Download and extract dataset SSCS

In [ ]:
sscs_download()

## 5 - Dataset Generator

In [ ]:
class SSCS_Sequence(tf.keras.utils.Sequence):
    def __init__(self, filenames):
        self.filenames = filenames
        self.splits_amount = 0
        self.splits_per_file = np.array([], dtype=np.intc)
        self.songs_dir = songs_dir
        self.split_size = SPLIT_SIZE

        for file in self.filenames:
            file_access = self.songs_dir + file + ".h5"
            f = h5py.File(file_access, 'r')
            file_shape = f['mix/table'].shape[0]
            df_batch_items = file_shape//self.split_size
            #if(file_shape/self.split_size > df_batch_items): df_batch_items += 1
            self.splits_per_file = np.append(self.splits_per_file, int(df_batch_items))
            self.splits_amount += df_batch_items
            f.close()
        
    def __len__(self):
        return self.splits_amount

    def __getitem__(self, idx):
        splits_past = idx
        idx_get = 0
        if(idx > 0):
            while(splits_past > 0):
                splits_past -= self.splits_per_file[idx_get]
                if(splits_past > 0):
                    idx_get += 1
        split_get = self.splits_per_file[idx_get] + splits_past
        
        if(split_get == self.splits_per_file[idx_get]):
            idx_get += 1
            split_get = 0

        file_access = self.songs_dir + self.filenames[idx_get] + ".h5"
        f = h5py.File(file_access, 'r')
        data_min = split_get * self.split_size
        data_max = data_min + self.split_size

        mix_in = np.transpose(pd.read_hdf(file_access, 'mix')[data_min:data_max]).to_numpy()
        s_out = np.transpose(pd.read_hdf(file_access, 'soprano')[data_min:data_max]).to_numpy()
        a_out = np.transpose(pd.read_hdf(file_access, 'alto')[data_min:data_max]).to_numpy()
        t_out = np.transpose(pd.read_hdf(file_access, 'tenor')[data_min:data_max]).to_numpy()
        b_out = np.transpose(pd.read_hdf(file_access, 'bass')[data_min:data_max]).to_numpy()

        mix_in = mix_in.reshape((mix_in.shape[0], mix_in.shape[1], 1))
        s_out = s_out.reshape((s_out.shape[0], s_out.shape[1], 1))
        a_out = a_out.reshape((a_out.shape[0], a_out.shape[1], 1))
        t_out = t_out.reshape((t_out.shape[0], t_out.shape[1], 1))
        b_out = b_out.reshape((b_out.shape[0], b_out.shape[1], 1))

        mix_in = tf.convert_to_tensor(mix_in, dtype=tf.float32)
        s_out = tf.convert_to_tensor(s_out, dtype=tf.float32)
        a_out = tf.convert_to_tensor(a_out, dtype=tf.float32)
        t_out = tf.convert_to_tensor(t_out, dtype=tf.float32)
        b_out = tf.convert_to_tensor(b_out, dtype=tf.float32)

        f.close()

        return mix_in, [s_out, a_out, t_out, b_out]

    def get_splits_per_file(self):
        return self.splits_per_file

In [ ]:
seq = SSCS_Sequence(sscs_get_split()[30:60])

In [ ]:
seq.__len__()

## 6 - Plot test

In [ ]:
mix, satb = seq.__getitem__(np.random.randint(0, seq.__len__()))

In [ ]:
sscs_plot(mix)

## 7 - Training VoasCNN

In [ ]:
ds = tf.data.Dataset.from_generator(SSCS_Sequence, np.float32, (360, 128), [sscs_get_split()[30:60]])

for elem in ds:
    print(elem.numpy())

In [ ]:
class VoasCrossentropy(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        log_y_pred = tf.math.log(y_pred)
        log_y_pred_comp = tf.math.log(1 - y_pred)
        aux1 = -tf.math.multiply_no_nan(x=log_y_pred, y=y_true)
        aux2 = -tf.math.multiply_no_nan(x=log_y_pred_comp, y=(1 - y_true))
        elements = aux1 + aux2
        return tf.reduce_mean(tf.reduce_sum(elements, range(tf.rank(y_pred))))

In [ ]:
voas_cnn = voas_cnn_model()
voas_cnn.compile(optimizer=Adam(learning_rate=2e-3),
                 loss=VoasCrossentropy(),
                 metrics=[Precision()])
#voas_cnn.summary()

In [ ]:
voas_cnn.fit(seq, batch_size=32, epochs=3)